In [1]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

from nltk.corpus import brown

In [2]:
print(brown.tagged_words()) 

[('The', 'AT'), ('Fulton', 'NP-TL'), ...]


In [3]:
# storing all the POS tags, words
tags = defaultdict()
words = defaultdict()
tag_words = defaultdict()
for tuple in brown.tagged_words(tagset='universal'):
    tag = tuple[1]
    word = tuple[0]
    if tag in tags: tags[tag] += 1
    else: tags[tag] = 1
        
    if word in words: words[word] += 1
    else: words[word] = 1
        
    if tag in tag_words:
        if word in tag_words[tag]: tag_words[tag][word] += 1
        else: tag_words[tag][word] = 1
    else:
        tag_words[tag] = defaultdict()
        tag_words[tag][word] = 1

In [4]:
# checking tag_words
word = "this"
tag = "DET"
if word in tag_words[tag]:
    print(tag_words[tag][word])

3965


In [5]:
# checking tags
for key,value in tags.items():
    print(key, value)

DET 137019
NOUN 275558
ADJ 83721
VERB 182750
ADP 144766
. 147565
ADV 56239
CONJ 38151
PRT 29829
PRON 49334
NUM 14874
X 1386


In [6]:
count = 0
for key,value in words.items():
    if count > 10: break
    count += 1
    print(key, value)

The 7258
Fulton 17
County 85
Grand 18
Jury 4
said 1943
Friday 60
an 3542
investigation 43
of 36080
Atlanta's 4


## Probabilities to compute: 
    P(word|tag) = count(word,tag)/count(all words,tag) -> emission probabilities
    P(tag1|tag2) = count(tag2,tag1)/count(tag2,tagx) -> transition probabilities
 


## Emission Probabilities

In [7]:
# computing emission probabilities
emission = defaultdict()
for tag in tags.keys():
    for word in words.keys():
        
        if tag not in emission:
            emission[tag] = defaultdict()
        count = 0
        if word in tag_words[tag]: count = tag_words[tag][word]
        emission[tag][word] = count/tags[tag]

In [8]:
# checking probabilities
tag = "DET"
word = "the"
emission[tag][word]

0.45767375327509324

## Computing bi-gram counts

In [9]:
sents = []
for lis in brown.tagged_sents(tagset='universal'):
    sents.append(lis)

print(len(sents))

57340


In [10]:
count = 0
for i,sent in enumerate(sents):
    if count > 10: break
    if len(sent) == 5: 
        print(i, sent)
        count += 1

187 [('``', '.'), ('Must', 'VERB'), ('solve', 'VERB'), ('problem', 'NOUN'), ("''", '.')]
655 [('The', 'DET'), ('hotel', 'NOUN'), ('owner', 'NOUN'), ('shrugged', 'VERB'), ('.', '.')]
676 [('Formula', 'NOUN'), ('is', 'VERB'), ('due', 'ADJ'), ('this', 'DET'), ('week', 'NOUN')]
962 [('Oak', 'NOUN'), ('Grove', 'NOUN'), ('(', '.'), ('special', 'ADJ'), (')', '.')]
981 [('--', '.'), ('emphasizes', 'VERB'), ('the', 'DET'), ('Virgin', 'NOUN'), ('birth', 'NOUN')]
1004 [("'", '.'), ('church', 'NOUN'), ('meets', 'VERB'), ('change', 'NOUN'), ("'", '.')]
1069 [('Seeks', 'VERB'), ('``', '.'), ('improved', 'VERB'), ('fielding', 'VERB'), ("''", '.')]
1074 [('Duren', 'NOUN'), (',', '.'), ('Sheldon', 'NOUN'), ('on', 'ADP'), ('hill', 'NOUN')]
1214 [('A', 'DET'), ('quick', 'ADJ'), ('touchdown', 'NOUN'), ('resulted', 'VERB'), ('.', '.')]
1402 [('It', 'PRON'), ('made', 'VERB'), ('him', 'PRON'), ('human', 'NOUN'), ('.', '.')]
1428 [('He', 'PRON'), ('had', 'VERB'), ('a', 'DET'), ('16', 'NUM'), ('.', '.')]


In [11]:
bigram = defaultdict()
for sent in sents:
    length = len(sent)
    for i in range(0, length-1):
        curr = sent[i][1]
        next_ = sent[i+1][1]
        bi = curr + '-' + next_
        if bi not in bigram: bigram[bi] = 1
        else: bigram[bi] += 1

In [12]:
# test bi-grams
print(bigram["NOUN-DET"])
print(tags["NOUN"])

4270
275558


In [13]:
# compute bi-gram counts for starting tag and next tag
for sent in sents:
    start_tag = sent[0][1]
    bi = "^-" + start_tag
    if bi not in bigram: bigram[bi] = 1
    else: bigram[bi] += 1

## Transition Probabilities

In [14]:
# check bi-grams of states which will be divided by count of initial state
transition = defaultdict()
for tag in tags.keys():
    for tag1 in tags.keys():
        if tag not in transition:
            transition[tag] = defaultdict()
        transition[tag][tag1] = bigram[tag+'-'+tag1]/tags[tag] 

In [15]:
# compute starting tag transition probability i.e P(X|^)
transition["^"] = defaultdict()
for tag in tags.keys():
     transition["^"][tag] = bigram["^"+"-"+tag]/len(sents)

In [16]:
# test transition
print(transition["DET"]["NOUN"])
print(transition["^"]["ADJ"])

0.6264678621213117
0.034339030345308684


In [17]:
print(len(words))
print(len(sents))

56057
57340


In [23]:
count = 0
for sent in sents:
    if count >= 3: break
    for tup in sent:
        if tup[1] == "X":
            sentence = ""
            for tupp in sent:
                sentence += tupp[0] + " "
            count += 1
            print(sentence, sent)
            break

There should even be no more bitter surprises in the UN General Assembly as to NATO members' votes , since a new ad hoc NATO committee has been set up so that in the future such topics as Angola will be discussed in advance .  [('There', 'PRT'), ('should', 'VERB'), ('even', 'ADV'), ('be', 'VERB'), ('no', 'DET'), ('more', 'ADJ'), ('bitter', 'ADJ'), ('surprises', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('UN', 'NOUN'), ('General', 'ADJ'), ('Assembly', 'NOUN'), ('as', 'ADP'), ('to', 'ADP'), ('NATO', 'NOUN'), ("members'", 'NOUN'), ('votes', 'NOUN'), (',', '.'), ('since', 'ADP'), ('a', 'DET'), ('new', 'ADJ'), ('ad', 'X'), ('hoc', 'X'), ('NATO', 'NOUN'), ('committee', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('set', 'VERB'), ('up', 'PRT'), ('so', 'ADP'), ('that', 'ADP'), ('in', 'ADP'), ('the', 'DET'), ('future', 'NOUN'), ('such', 'ADJ'), ('topics', 'NOUN'), ('as', 'ADP'), ('Angola', 'NOUN'), ('will', 'VERB'), ('be', 'VERB'), ('discussed', 'VERB'), ('in', 'ADP'), ('advance', 'NOUN'), ('.', 

In [26]:
count = 0
for sent in sents:
    if count >= 3: break
    for tup in sent:
        if tup[0] == "cease-fire" and (tup[1] == "X"):
            sentence = ""
            for tupp in sent:
                sentence += tupp[0] + " "
            count += 1
            print(sentence, sent)
            break

In [18]:
parameters = {"tags": tags, "transition": transition, "emission": emission, "vocab": words}

## Viterbi Algo

In [25]:
import copy
import time

In [42]:
class TreeNode:
    def __init__(self, tag, prob, parent):
        self.tag = tag
        self.prob = prob
        self.tags = []
        self.parent = parent
        

class Viterbi:
    
    def find_tags(self, imp_nodes):
        t1 = time.time()
        max_node = None
        for tag,tag_node in imp_nodes.items():
            #print("tag ", tag, tag_node.prob, tag_node.tags[0].prob)
            if max_node is None or max_node.tags[0].prob < tag_node.tags[0].prob: 
                max_node = tag_node
      
        tags = []
        while(max_node is not None):
            tags.insert(0, max_node.tag)
            max_node = max_node.parent
      
        print("find tags takes", time.time()-t1)
        return tags
      
    def compute_states(self, sent, parameters):
        sent = sent.strip()
        tokens = sent.split(' ')
      
        root = TreeNode("^", 1, None)
        imp_nodes = defaultdict()
      
        # create first level of tree
        for tag in parameters["tags"].keys():
            node = TreeNode(tag, parameters["transition"]["^"][tag], root)
            root.tags.append(node)
            imp_nodes[tag] = node
      
      
        temp_best = defaultdict()
        #print(tokens)
        t1 = time.time()
        for i,token in enumerate(tokens):
            temp_best = defaultdict()
            for tag,tag_node in imp_nodes.items():
              # compute every tag for this node
                if i == len(tokens)-1:
                    if token in parameters["vocab"]:
                          emission = parameters["emission"][tag][token]
                    else:
                      emission = 0.001
                    transition = parameters["transition"][tag]["."]
                    new_prob = tag_node.prob*emission*transition
                    child = TreeNode(".", new_prob, tag_node)
                    tag_node.tags.append(child)
                else: 
                    for child_tag in parameters["tags"].keys():
                        if token in parameters["vocab"]:
                          emission = parameters["emission"][tag][token]
                        else:
                          emission = 0.001
                        transition = parameters["transition"][tag][child_tag]
                        new_prob = tag_node.prob*emission*transition
                        child = TreeNode(child_tag, new_prob, tag_node)
                        tag_node.tags.append(child)
                        if child_tag in temp_best:
                            if new_prob > temp_best[child_tag].prob:
                                temp_best[child_tag] = child
                        else:
                            temp_best[child_tag] = child
          
            if i < len(tokens) - 1:
                for tag in parameters["tags"].keys(): 
                  imp_nodes[tag] = temp_best[tag]
#                   print("token : ",token," tag: ", tag, imp_nodes[tag].prob)
      
        print("tokens for loop took", time.time()-t1)
        return self.find_tags(imp_nodes)
        
        

obj = Viterbi()
sent = "Why thats what gave me the feeling gave me asitwere the spirit the demoniac evil spirit of this whole affair"
tokens = [i[0] for i in sents[450]]
#sent = " ".join(tokens)
t1 = time.time()
states = obj.compute_states(sent, parameters) 
print("overall time is ", time.time()-t1)

tokens for loop took 0.017392396926879883
find tags takes 1.8358230590820312e-05
overall time is  0.019480228424072266


In [39]:
states

['^',
 'ADV',
 'ADP',
 'DET',
 'VERB',
 'PRON',
 'DET',
 'NOUN',
 'VERB',
 'PRON',
 'VERB',
 'DET',
 'NOUN',
 'DET',
 'ADJ',
 'ADJ',
 'NOUN',
 'ADP',
 'DET',
 'ADJ',
 'NOUN']